In [1]:
!python --version

Python 3.10.12


In [ ]:
MODEL_NAME = 'meta-llama/Llama-3.2-1B-Instruct'
TASK = 'text-generation'

ITER_START = 5
ITER_TRIALS = 1

# activate when feedback_results is saved in current iter
SKIP_FEEDBACK = True
# running test
RUN_LIMITED_TEST = False

FILEDIR_PREFIX = "/kaggle/working/llama3-1B/iter_"
DATASET_FILEPATH_PREFIX = "/kaggle/input/flan-t5/llama3-1B/iter_"

HF_TOKEN = "HF_TOKEN" # input huggingface token

# Load Dependencies

In [3]:
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm  # For progress bars

In [4]:
!pip install -qU transformers
!pip install -qU accelerate
!pip install -qU bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.8 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, pipeline, BitsAndBytesConfig
from transformers.pipelines.pt_utils import KeyDataset
import torch
import numpy as np
import os
import math
from datasets import concatenate_datasets, Dataset
from tqdm import tqdm
import time

In [6]:
# setting env vars
set_seed(1234)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
import os

os.environ["HF_TOKEN"] = HF_TOKEN

In [8]:
!huggingface-cli login --token $HF_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `Auth` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Prep Functions

In [9]:
def query_model(data, column_to_prompt, num_new_tokens = 200, batch_size=40):
    print(f"...processing in batches of {batch_size}")
    responses = []
    
    for i in range(0, len(data[column_to_prompt]), batch_size):
        batch = data[column_to_prompt][i:i+batch_size]
        # print(f"...processing batch starting with #{i}. Number of examples: {len(batch)}")

        formatted_input = [tokenizer.apply_chat_template(
                                convo,
                                tokenize=False,
                                add_generation_prompt=True
                            ) for convo in batch
                          ]
        
        model_inputs = tokenizer(formatted_input, padding=True, truncation=True, return_tensors="pt").to(model.device)
    
        with torch.no_grad():
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=num_new_tokens,
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        responses += response
    
        # cleaning up memory
        del model_inputs
        del generated_ids
        torch.cuda.empty_cache()

    return responses

In [10]:
def append_to_all_answers(example):
    example['all_answers'].append(example['predicted_answer'])

    return example

## Feedback functions

In [11]:
def generate_feedback_prompts(example, template, instruction, fs):
    prompt = []

    prompt.append({"role": "system",
                  "content": instruction})
    
    for fs_example in fs:
        prompt.append({"role": "user",
                      "content": (template.invoke(fs_example).text)})
        prompt.append({"role": "assistant",
                      "content": fs_example['predicted_answer_feedback']})
    
    prompt.append({"role": "user",
                  "content": (template.invoke(dict(example)).text)})
     
    example['feedback_prompt'] = prompt

    return example

In [12]:
def prompt_for_feedback(ds):
    start = time.time()
    output_100 = query_model(ds.select(range(100)), "feedback_prompt", 200)
    end = time.time()
    print(f"Processing 100 records took {end-start} secs. Estimated completion: {(len(ds) / 100)*(end-start)} secs [{((len(ds) / 100)*(end-start)) / 60} mins]")
    output_next = query_model(ds.select(range(100, len(ds))), "feedback_prompt", 200)
    print('Done. Combining outputs')
    output = output_100 + output_next
    # cleaned_output = [out[0]['generated_text'][2]['content'].strip() for out in output]
    cleaned_output = [out.strip() for out in output]

    if 'predicted_answer_feedback' in ds.column_names:
        ds = ds.remove_columns(['predicted_answer_feedback'])

    ds = ds.add_column('predicted_answer_feedback', cleaned_output)
    return ds

In [13]:
def generate_feedback(dataset, feedback_instruction, feedback_prompt_template, feedback_fs, filedir_prefix_saveto):
    # generating feedback prompts
    dataset['train'] = dataset['train'].map(generate_feedback_prompts, 
                                            fn_kwargs={'template':feedback_prompt_template,
                                                       'instruction': feedback_instruction,
                                                       'fs': feedback_fs})
    dataset['train'].to_parquet(f"{filedir_prefix_saveto}feedback_prompts.parquet")
    # obtaining feedback
    dataset['train'] = prompt_for_feedback(dataset['train'])

    return dataset

## Refine functions

In [14]:
def generate_refine_prompts(example, template, instruction, fs):
    prompt = []
    
    prompt.append({"role": "system",
                  "content": instruction})

    for fs_example in fs:
        prompt.append({"role": "user",
                      "content": (template.invoke(fs_example).text)})
        prompt.append({"role": "assistant",
                      "content": fs_example['refined_answer']})
    
    prompt.append({"role": "user",
                  "content": (template.invoke(dict(example)).text)})
     
    example['refine_prompt'] = prompt

    return example

In [15]:
def prompt_for_refinement(ds):
    start = time.time()
    output_100 = query_model(ds.select(range(100)), "refine_prompt", 50)
    end = time.time()
    print(f"Processing 100 records took {end-start} secs. Estimated completion: {(len(ds) / 100)*(end-start)} secs [{((len(ds) / 100)*(end-start)) / 60} mins]")
    output_next = query_model(ds.select(range(100, len(ds))), "refine_prompt", 50)
    print('Done. Combining outputs')
    output = output_100 + output_next
    # cleaned_output = [out[0]['generated_text'][2]['content'].strip() for out in output]
    cleaned_output = [out.strip() for out in output]

    if 'predicted_answer' in ds.column_names:
        ds = ds.remove_columns(['predicted_answer'])

    ds = ds.add_column('predicted_answer', cleaned_output)
    
    return ds

In [16]:
def generate_refined(dataset, refine_instruction, refine_prompt_template, refine_fs, filedir_prefix_saveto):
    # generating refine prompts
    dataset['train'] = dataset['train'].map(generate_refine_prompts, 
                                            fn_kwargs={'template':refine_prompt_template,
                                                      'instruction': refine_instruction,
                                                       'fs': refine_fs})
    dataset['train'].to_parquet(f"{filedir_prefix_saveto}refine_prompts.parquet")
    # obtaining refined prompts
    dataset['train'] = prompt_for_refinement(dataset['train'])
    return dataset

# Load Dataset

In [17]:
from datasets import load_dataset

In [18]:
static_fs = [
    {
        'context': "Dominic's education at Palencia gave him the knowledge he needed to overcome the Manicheans. With charity, the other concept that most defines the work and spirituality of the order, study became the method most used by the Dominicans in working to defend the Church against the perils that hounded it, and also of enlarging its authority over larger areas of the known world. In Dominic's thinking, it was impossible for men to preach what they did not or could not understand. When the brethren left Prouille, then, to begin their apostolic work, Dominic sent Matthew of Paris to establish a school near the University of Paris. This was the first of many Dominican schools established by the brethren, some near large universities throughout Europe.",
        'question': "What method was not used by the Dominicans in working to defend the Church against peril?",
        'predicted_answer': "Charity",
        'predicted_answer_feedback': """To assess the quality of the answer, we assign a score to each trait:
1. Relevancy - how well does the answer address the question: Very well. Charity is a method that feasibly could have been used to defend the Church against peril. 10/10
2. Consistency - does the answer match what is said in the context: No, the context explicitly states that charity and study were both used by the Dominicans in working to defend the Church against peril. The answer contradicts the context. 0/10
3. Brevity - does the answer only include the answer and nothing else: Yes, the answer is concise and to the point and only includes the answer to the question. 10/10""",
        'refined_answer': "Cannot be determined."
    },
#     {
#         'context': "In Canada, the traditional ceremony for granting assent in parliament was regularly used until the 21st century, long after it had been discontinued in the United Kingdom and other Commonwealth realms. One result, conceived as part of a string of royal duties intended to demonstrate Canada's status as an independent kingdom, was that King George VI personally assented to nine bills of the Canadian parliament during the 1939 royal tour of Canada\u201485 years after his great-grandmother, Queen Victoria, had last granted royal assent personally in the United Kingdom. Under the Royal Assent Act 2002, however, the alternative practice of granting assent in writing, with each house being notified separately ( the Speaker of the Senate or a representative reads to the senators the letters from the governor general regarding the written declaration of Royal Assent), was brought into force. As the act also provides, royal assent is to be signified\u2014by the governor general, or, more often, by a deputy, usually a Justice of the Supreme Court, at least twice each calendar year: for the first appropriation measure and for at least one other act, usually the first non-appropriation measure passed. However, the act provides that a grant of royal assent is not rendered invalid by a failure to employ the traditional ceremony where required.",
#         'question': "When did Canada finally cease to use the traditional ceremony for granting assent as regular practice?",
#         'predicted_answer': "21st century",
#         'predicted_answer_feedback': """To assess the quality of the answer, we assign a score to each trait:
# 1. Relevancy - how well does the answer address the question: Very well. The answer provides a date which would answer the question for when did Canada finally cease to use the traditional ceremony for granting assent as regular practice. 10/10
# 2. Consistency - does the answer match what is said in the context: Yes, the context explicitly states that the traditional ceremony was regularly used until the 21st century. 10/10
# 3. Brevity - does the answer only include the answer and nothing else: Yes, the answer is concise and to the point and only includes the answer to the question. 10/10""",
#         'refined_answer': "21st century"
#     },
    {
        'context': "Scholars in the United Kingdom and the United States developed somewhat different versions of cultural studies after the late 1970s. The British version of cultural studies had originated in the 1950s and 1960s, mainly under the influence first of Richard Hoggart, E. P. Thompson, and Raymond Williams, and later that of Stuart Hall and others at the Centre for Contemporary Cultural Studies at the University of Birmingham. This included overtly political, left-wing views, and criticisms of popular culture as \"capitalist\" mass culture; it absorbed some of the ideas of the Frankfurt School critique of the \"culture industry\" (i.e. mass culture). This emerges in the writings of early British cultural-studies scholars and their influences: see the work of (for example) Raymond Williams, Stuart Hall, Paul Willis, and Paul Gilroy.",
        'question': "When did the British version of cultural studies emerge?",
        'predicted_answer': "The British version of cultural studies emerged in the 1950s and 1960s.",
        'predicted_answer_feedback': """To assess the quality of the answer, we assign a score to each trait:
1. Relevancy - how well does the answer address the question: Very well. The answer provides dates which would answer the question for when the British version of cultural studies emerged. 10/10
2. Consistency - does the answer match what is said in the context: Yes, the context explicitly states that those "cultural studies had originated in the 1950s and 1960s". 10/10
3. Brevity - does the answer only include the answer and nothing else: No, the answer includes the response to the question and a part of the question. The latter part can be omitted for brevity. 5/10""",
        'refined_answer': "1950s and 1960s"
    },
    {
        'context': "The very large and ornate School Hall and School Library (by L K. Hall) were erected in 1906-8 across the road from Upper School as the school's memorial to the Etonians who had died in the Boer War. Many tablets in the cloisters and chapel commemorate the large number of dead Etonians of the Great War. A bomb destroyed part of Upper School in World War Two and blew out many windows in the Chapel. The college commissioned replacements by Evie Hone (1949–52) and by John Piper and Patrick Reyntiens (1959 onwards).",
        'question': "Who was one of the prominent Etonians who died in the Great War?",
        'predicted_answer': "Etonians who die in the Great War",
        'predicted_answer_feedback': """To assess the quality of the answer, we assign a score to each trait:
1. Relevancy - how well does the answer address the question: Not well. The answer repeats the question without providing a clear response. Etonians who die in the Great War cannot be one of the prominent Etonians who died in the Great War. 0/10
2. Consistency - does the answer match what is said in the context: No. The context has no mention of the prominent Etonians who died in the Great War, only that Etonians died in the Great War. 0/10
3. Brevity - does the answer only include the answer and nothing else: It says Etonians who die in the Great War are one of the prominent Etonians who died in the Great War, which is redundant. The entirety of the answer can be omitted. 0/10""",
        'refined_answer': "Cannot be determined."
    },
    
]

In [19]:
feedback_instruction = """We want to iteratively improve the provided answers. To help improve, score each answer on a scale of 1 to 10 on the traits:
1) Relevancy - how well does the answer address the question. 10 out of 10 means the answer directly responds to the question. 5 out of 10 means the answer partially responds to the question but not entirely. 0 out of 10 means the answer does not respond to the question.
2) Consistency - does the answer match what is said in the context. 10 out of 10 means the answer of the question is explicitly stated in the context. 5 out of 10 means the answer to the question is not supported or proven by the context. 0 out of 10 means the answer to the question contradicts what is written in the context. 
3) Brevity - does the answer only include the answer and nothing else. 10 out of 10 means the answer only contains the response to the question. 5 out of 10 means the answer contains the response to the question and a portion of the question. 0 out of 10 means the answer contains unnecessarily details that can be omitted.

Note that if the answer to the question cannot be determined from the context, the answer should say 'Cannot be determined'."""

feedback_template = """Context:
{context}

Question:
{question}

Answer:
{predicted_answer}

Scores:
"""

feedback_prompt_template = PromptTemplate.from_template(feedback_template)

In [20]:
refine_instruction = "Use the feedback to improve the answer to the question in such a way that obtains a perfect score for each evaluated trait. Output only the refined answer and nothing else. If the answer achieved a perfect score, simply return the initial answer. Note that if the answer cannot be determined from the context, only say 'Cannot be determined' and nothing else."

refine_template = """Context:
{context}

Question:
{question}

Answer:
{predicted_answer}

Feedback:
{predicted_answer_feedback}

Refined Answer:
"""

refine_prompt_template = PromptTemplate.from_template(refine_template)

In [21]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_quant_type="nf4",             
    bnb_4bit_use_double_quant=True,       
)

In [22]:
print(f"LOADING IN MODEL: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
    cache_dir="/kaggle/working/cache",
    device_map="auto",
    # use_sliding_window=False,
    # max_memory={"cuda:0": "16GB", "cuda:1": "16GB"},
    trust_remote_code=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = 128001
tokenizer.padding_side='left'

LOADING IN MODEL: meta-llama/Llama-3.2-1B-Instruct


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [23]:
print(f"STARTING PIPELINE")
# generator = pipeline(TASK, model=model, tokenizer = tokenizer)

STARTING PIPELINE


In [24]:
for iter_i in range(ITER_START, ITER_START + ITER_TRIALS):
    print(f"RUNNING ITERATION {iter_i}")
    if iter_i == ITER_START:
        # loading appropriate dataset from trial 0
        dataset_filepath = f"{DATASET_FILEPATH_PREFIX}{iter_i-1}/results.parquet"
    else:
        dataset_filepath = f"{FILEDIR_PREFIX}{iter_i-1}/results.parquet"
    
    dataset = load_dataset("parquet", data_files=dataset_filepath)

    if RUN_LIMITED_TEST:
        dataset['train'] = dataset['train'].select(range(150))
        print("Running a test: computing 150 examples.")
        
    filedir_prefix_saveto = f"{FILEDIR_PREFIX}{iter_i}/"
    # feedback
    print("- Generating feedback...")
    if iter_i == ITER_START and SKIP_FEEDBACK:
        dataset = load_dataset("parquet", data_files=f"{DATASET_FILEPATH_PREFIX}{iter_i}/results_feedback.parquet")
        print('SKIPPED: Retrieved feedback responses from dataset.')
    else:
        dataset = generate_feedback(dataset,
                                    feedback_instruction,
                                    feedback_prompt_template,
                                    static_fs,
                                    filedir_prefix_saveto)
        
        dataset['train'].to_parquet(f"{filedir_prefix_saveto}results_feedback.parquet")
    # refine
    print("- Refining answer...")
    dataset = generate_refined(dataset,
                              refine_instruction,
                              refine_prompt_template,
                              static_fs,
                              filedir_prefix_saveto)
    
    print("- Adding to all answers...")
    dataset["train"] = dataset["train"].map(append_to_all_answers)
    dataset['train'].to_parquet(f"{filedir_prefix_saveto}results.parquet")
    
    print('Done.')

RUNNING ITERATION 5


Generating train split: 0 examples [00:00, ? examples/s]

- Generating feedback...


Generating train split: 0 examples [00:00, ? examples/s]

SKIPPED: Retrieved feedback responses from dataset.
- Refining answer...


...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

...processing in batches of 40
Processing 100 records took 171.36115288734436 secs. Estimated completion: 8568.057644367218 secs [142.80096073945364 mins]
...processing in batches of 40
Done. Combining outputs
- Adding to all answers...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Done.
